This notebook is about creating concatenated interaction events from the source video and fly and ball positions

## Imports

In [7]:
import h5py
from scipy import signal
import numpy as np
import pandas as pd
from pathlib import Path
import sys
sys.path.insert(0, "../..")
# from Utilities.Utils import *
# from Utilities.Processing import *
import cv2
from datetime import timedelta

import os
os.environ["IMAGEMAGICK_BINARY"] = "/usr/bin/convert"  # Replace with your actual path



## Functions

These are the same as FlyBall_interactions.py. In the end I'll probably make sure these are inherited from the same place.

In [4]:

def savgol_lowpass_filter(data, window_length, polyorder):
    # Apply the Savitzky-Golay filter
    y = signal.savgol_filter(data, window_length, polyorder)
    return y

def extract_coordinates(h5_file):
    with h5py.File(h5_file, "r") as f:
        locs = f["tracks"][:].T
        y = locs[:, :, 1, :].squeeze()
        x = locs[:, :, 0, :].squeeze()
    return x, y


def replace_nans_with_previous_value(arr):
    # Find the indices of the NaN values
    nan_indices = np.where(np.isnan(arr))

    # Replace the NaN values with the previous value
    for i in nan_indices[0]:
        arr[i] = arr[i - 1]


def extract_interaction_events(ballpath, flypath, Thresh=80, min_time=60):
    """
    Extracts the interaction events from the ball and fly paths.

    Parameters
    ----------
    ballpath : str
        The path to the ball path file.
        flypath : str
        The path to the fly path file.
        Thresh : int
        The threshold distance between the ball and fly.
        min_time : int
        The minimum duration of an interaction event.

        Returns
        -------
        interaction_events : list
        A list of DataFrames containing the interaction events.
    """
    xball, yball = extract_coordinates(ballpath.as_posix())
    xfly, yfly = extract_coordinates(flypath.as_posix())

    # Replace NaNs in yball
    replace_nans_with_previous_value(yball)

    # Replace NaNs in xball
    replace_nans_with_previous_value(xball)

    # Replace NaNs in yfly
    replace_nans_with_previous_value(yfly)

    # Replace NaNs in xfly
    replace_nans_with_previous_value(xfly)

    # Combine the yball and yfly arrays into a single 2D array
    data = np.stack((yball, yfly), axis=1)

    # Create a pandas DataFrame from the data
    df = pd.DataFrame(data, columns=["yball", "yfly"])

    df["yball_smooth"] = savgol_lowpass_filter(df["yball"], 221, 1)
    df["yfly_smooth"] = savgol_lowpass_filter(df["yfly"], 221, 1)
    df = df.assign(Frame=df.index + 1)
    df["time"] = df["Frame"] / 30

    # Compute the difference between the yball and yfly positions smoothed
    df["dist"] = df["yfly_smooth"] - df["yball_smooth"]

    # Locate where the distance is below the threshold
    df["close"] = df["dist"] < Thresh

    df = df.reset_index()

    # Find the start and end indices of streaks of True values in the 'close' column
    df["block"] = (df["close"].shift(1) != df["close"]).cumsum()
    events = (
        df[df["close"]]
        .groupby("block")
        .agg(start=("index", "min"), end=("index", "max"))
    )

    # Store the interaction events as separate DataFrames
    interaction_events = [
        df.loc[start:end]
        for start, end in events[["start", "end"]].itertuples(index=False)
    ]

    # remove events that are less than min_time frames long
    interaction_events = [
        event for event in interaction_events if len(event) >= min_time
    ]

    # event_times = [(df["time"].min(), df["time"].max()) for df in interaction_events]

    return interaction_events

In [5]:
# Example usage:
ballpath = Path(
    "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos/230721_Feedingstate_4_PM_Videos_Tracked/arena5/corridor3/corridor3_tracked.000_corridor3.analysis.h5"
)

flypath = Path(
    "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos/230721_Feedingstate_4_PM_Videos_Tracked/arena5/corridor3/flytrack.000_corridor3.analysis.h5"
)

vidpath = Path(
    "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos/230721_Feedingstate_4_PM_Videos_Tracked/arena5/corridor3/corridor3.mp4"
)

interaction_events = extract_interaction_events(ballpath, flypath)

OutFolder = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Grids")

In [12]:
import cv2
import numpy as np

def check_yball_variation(event_df, threshold=10):
    yball_segment = event_df['yball_smooth']
    variation = yball_segment.max() - yball_segment.min()
    return variation > threshold

clips = []
# Assuming interaction_events is a list of dataframes
for i, event_df in enumerate(interaction_events):
    start_frame, end_frame = event_df['Frame'].min(), event_df['Frame'].max()
    start_time, end_time = event_df['time'].min(), event_df['time'].max()  # Assuming 'time' is in seconds
    start_time_str = str(timedelta(seconds=int(start_time)))
    # Load the video
    cap = cv2.VideoCapture(str(vidpath))

    # Get the video's width, height, and frames per second (fps)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Be sure to use lower case
    
    clip_path = OutFolder.joinpath(f"output_{i}.mp4").as_posix()
    
    out = cv2.VideoWriter(clip_path, fourcc, fps, (height, width))  # Note that width and height are swapped

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    for _ in range(start_frame, end_frame):
        ret, frame = cap.read()
        if not ret:
            break

        # Rotate frame 90 degrees clockwise
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

        # Write some Text
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = f"Event:{i+1} - start:{start_time_str}"
        font_scale = width / 150
        thickness = int(4 * font_scale)
        text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
        
        # Position the text at the top center of the frame
        text_x = (frame.shape[1] - text_size[0]) // 2
        text_y = 25
        cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)

        # Check if yball value varies more than threshold
        if check_yball_variation(event_df):  # You need to implement this function
            # Add red dot to segment
            dot = np.zeros((10, 10, 3), dtype=np.uint8)
            dot[:, :, 0] = 0
            dot[:, :, 1] = 0
            dot[:, :, 2] = 255
            dot = cv2.resize(dot, (20, 20))
            
            # Position the dot right next to the text at the top of the frame
            dot_x = text_x + text_size[0] + 10  # Position the dot right next to the text with a margin of 10
            
            # Adjusted position for dot_y to make it slightly higher
            dot_y_adjustment_factor = 1.2  
            dot_y = text_y - int(dot.shape[0] * dot_y_adjustment_factor) + text_size[1] // 2
            
            frame[dot_y:dot_y+dot.shape[0], dot_x:dot_x+dot.shape[1]] = dot

        # Write the frame into the output file
        out.write(frame)

    # Release everything when done
    cap.release()
    out.release()
    
    clips.append(clip_path)

cv2.destroyAllWindows()


In [15]:
# Define the codec and create a VideoWriter object for the final video
out = cv2.VideoWriter(OutFolder.joinpath('final_output.mp4').as_posix(), fourcc, fps, (height, width)) # Note that width and height are swapped

# Assuming 'clips' is a list of paths to the clip files
for clip_path in clips:
    cap = cv2.VideoCapture(clip_path)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Write the frame into the final output file
        out.write(frame)

    cap.release()

# Release the final output file when done
out.release()


In [7]:
OutFolder.joinpath("output.mp4")

PosixPath('/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Grids/output.mp4')

In [4]:
import cv2
import numpy as np

def check_yball_variation(event_df, threshold=10):
    yball_segment = event_df['yball_smooth']
    variation = yball_segment.max() - yball_segment.min()
    return variation > threshold

# Load the video
cap = cv2.VideoCapture(str(vidpath))

# Get a frame from the video
cap.set(cv2.CAP_PROP_POS_FRAMES, 100)  # Change this to any frame number you want to test
ret, frame = cap.read()

if not ret:
    print("Failed to read frame from video")
else:
    # Rotate frame 90 degrees clockwise
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

    # Write some Text
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = "Test Text"
    font_scale = 1.0
    thickness = 2

    # Calculate the size of the text box
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    
    # Position the text at the top center of the frame
    text_x = (frame.shape[1] - text_size[0]) // 2
    text_y = 50

    # Add the text to the frame
    cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)

    # Create a red dot
    dot = np.zeros((10, 10, 3), dtype=np.uint8)
    dot[:, :, 0] = 0
    dot[:, :, 1] = 0
    dot[:, :, 2] = 255
    dot = cv2.resize(dot, (20, 20))
    
    # Position the dot right next to the text at the top of the frame
    dot_x = text_x + text_size[0] + 10  # Position the dot right next to the text with a margin of 10
    
    # Adjusted position for dot_y to make it slightly higher
    dot_y_adjustment_factor = 1.5  
    dot_y = text_y - int(dot.shape[0] * dot_y_adjustment_factor) + text_size[1] // 2
    
    # Add the dot to the frame
    frame[dot_y:dot_y+dot.shape[0], dot_x:dot_x+dot.shape[1]] = dot

# Display the frame
cv2.imshow('Test Frame', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

cap.release()


: 